In [ ]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')


### Merging group 1 CSVs

In [ ]:
working_multi_names = load_csv_dataset('CSVs of edge cases - for F/DONE/from group 1 - ad hoc fixes complete/working_multiple_names_combined.csv')
working_multi_names = working_multi_names.drop(columns={'Unnamed: 0'})
working_multi_names

In [ ]:
match_date_multi_guys_matches = load_csv_dataset('CSVs of edge cases - for F/DONE/from group 1 - ad hoc fixes complete/match_date_multi_guys_MATCHES.csv')
cyrillic_group_matches = load_csv_dataset('CSVs of edge cases - for F/DONE/from group 1 - ad hoc fixes complete/cyrillic_guys_multi_found_matches.csv')
online_group_Matches = load_csv_dataset('CSVs of edge cases - for F/DONE/from group 1 - ad hoc fixes complete/Online_lookup_required_multi_names_guys.csv')
used_lg_dataset_group = load_csv_dataset('CSVs of edge cases - for F/DONE/from group 1 - ad hoc fixes complete/Used_Large_Dataset_Multiple_Name_Dudes.csv')


In [ ]:
merged_df = pd.concat([working_multi_names, match_date_multi_guys_matches, cyrillic_group_matches, online_group_Matches, used_lg_dataset_group])
merged_df = merged_df.reset_index()
merged_df = merged_df.drop(columns={'index', 'Unnamed: 0'})

merged_df.at[41, 'Names_Found'] = '[\'Sohel Rana\', \'Mohamed Sohel Rana\']'
merged_df.at[60, 'Names_Found'] = '[\'Sohel Rana\', \'Mohamed Sohel Rana\']'
merged_df.at[83, 'Names_Found'] = '[\'Weston Mckennie\']'
merged_df.at[574, 'Names_Found'] = '[\'Sohel Rana\', \'Mohamed Sohel Rana\']'
merged_df.at[583, 'Names_Found'] = '[\'Sohel Rana\', \'Mohamed Sohel Rana\']'
merged_df.at[610, 'Names_Found'] = 'Alfred Finnbogason'
merged_df.at[1007, 'Names_Found'] = 'Brian Rodríguez'
merged_df.at[1012, 'Names_Found'] = 'Juan Fernando Quintero'
merged_df.at[1024, 'Names_Found'] = 'Brian Rodríguez'
merged_df.at[3289, 'Names_Found'] = '[\'Sohel Rana\', \'Mohamed Sohel Rana\']'
merged_df.at[3294, 'Names_Found'] = '[\'Sohel Rana\', \'Mohamed Sohel Rana\']'
merged_df.at[5491, 'Nationality'] = 'Cuba'
merged_df.at[5491, 'Nationality'] = 'CU'
merged_df.at[5491, 'Names_Found'] = 'Marcel Hernández'
merged_df.at[5782, 'Names_Found'] = 'Nigel Charles'

## Handle cases where mandem both / all played

In [ ]:
number_of_changes = 0
for i in range(0, 5957):
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = merged_df.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']
    names_found_string = row['Names_Found']
    if '‘' in names_found_string:
        names_found_string = names_found_string.replace('‘', "\'")
        names_found_string = names_found_string.replace('’', "\'")
        #print(names_found_string)
    original_jersey = row['ORIGINAL JERSEY']
    nationality = row['Nationality']
    if type(names_found_string) == str:
        if names_found_string.startswith('['):
            names_found = eval(names_found_string)
        else:  
            names_found = list(names_found_string)

    rows_matching_this_one = merged_df[(merged_df['Match'] == match) & 
                                       (merged_df['Date'] == date) &
                                       (merged_df['ORIGINAL JERSEY'] == original_jersey) &
                                       (merged_df['Nationality'] == nationality) &
                                       (merged_df['Names_Found'] == row['Names_Found'])]
    num_identical_rows = len(rows_matching_this_one)
    num_names_this = len(names_found)

    if num_names_this != 1:
        if num_identical_rows == num_names_this:
            matching_indexes = list(rows_matching_this_one.index)
            num = 0
            number_of_changes += 1
            for matching_index in matching_indexes:
                print(f'index is {matching_index}, num is {num}')
                merged_df.at[matching_index, 'Names_Found'] = names_found[num]
                print(f'added {names_found[num]} to row {matching_index}')
                num += 1
                #remove the name from names_found after adding it 
            #get all the indexes of the rows matching this one 

## Handle cases where the cell has the same name more than once

In [ ]:
def are_all_same(names_found):
    if not names_found:
        return False  # If the list is empty, technically all elements are the same, but I assume this is not what you want
    first_item = names_found[0]
    return all(item == first_item for item in names_found)

for i in range(0, 5957):
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = merged_df.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']
    names_found_string = row['Names_Found']
    if '‘' in names_found_string:
        names_found_string = names_found_string.replace('‘', "\'")
        names_found_string = names_found_string.replace('’', "\'")

    original_jersey = row['ORIGINAL JERSEY']
    nationality = row['Nationality']
    if type(names_found_string) == str:
        if names_found_string.startswith('['):
            names_found = eval(names_found_string)
        else:  
            names_found = list(names_found_string)
    if len(names_found) >= 2:
        if are_all_same(names_found):
            print("All items in the list are the same.", names_found[0])
            merged_df.at[i, 'Names_Found'] = names_found[0]

In [ ]:
merged_df.to_csv('Group1_fixes_complete.csv')

## Match Date Filtering, group 1 (DONE)

In [ ]:
group_1_fixes_complete_csv = load_csv_dataset('Group 2 (for T) - Find their $ Info/Group1_fixes_complete.csv')


In [ ]:
def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    


In [ ]:
instances = 0
for i in range(5750, 5957):
    names_found = []
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = group_1_fixes_complete_csv.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']
    names_found_string = row['Names_Found']
    if '‘' in names_found_string:
        names_found_string = names_found_string.replace('‘', "\'")
        names_found_string = names_found_string.replace('’', "\'")

    original_jersey = row['ORIGINAL JERSEY']
    nationality = row['Nationality']
    if type(names_found_string) == str:
        if names_found_string.startswith('['):
            names_found = eval(names_found_string)
        else: 
            0==0 
            #names_found = list(names_found_string)
    if len(names_found) >= 2:
        print("Multiple names remaining", i, names_found, match, date, nationality)
        filtered_names_match_date = filter_using_date_of_match(names_found, nationality, date, False)[1]
        
        if type(filtered_names_match_date) == str:
            print(f'{i} -- successfully filtered to just {filtered_names_match_date}')
            #correct_names = filtered_names_match_date
            status = 'Success'
            group_1_fixes_complete_csv.at[i, 'Names_Found'] = filtered_names_match_date
        else:
            #multi_list = filtered_names_match_date
            print(f'{i} -- after filtering the remaining list is {filtered_names_match_date}')
            group_1_fixes_complete_csv.at[i, 'Status'] = 'MULTIPLE'
            group_1_fixes_complete_csv.at[i, 'Names_Found'] = filtered_names_match_date

        #try doing a match date filter first of all. 

        #FOR STATUS == MULTIPLE
            #check if there is another row with the same Nationality, Match, Date in the working dataset.
                #and that one of the names in these rows is in the Names_Found

                #for all such rows you find , remove that from the list of names in this row

            #when ur done checking that count how many names remain
                #if 1, leave it. 
                #if 2+, ? 
    else:
        0==0 #add the bread info for this guy.
#print(instances)

### Again handle stuff where mandem both played (in this dataset)

In [ ]:
number_of_changes = 0
for i in range(0, 5957):
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = group_1_fixes_complete_csv.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']
    names_found_string = row['Names_Found']
    if '‘' in names_found_string:
        names_found_string = names_found_string.replace('‘', "\'")
        names_found_string = names_found_string.replace('’', "\'")
        #print(names_found_string)
    original_jersey = row['ORIGINAL JERSEY']
    nationality = row['Nationality']
    if type(names_found_string) == str:
        if names_found_string.startswith('['):
            names_found = eval(names_found_string)
        else:  
            names_found = list(names_found_string)
    

    if type(row['Names_Found']) == list:
        rows_matching_this_one = group_1_fixes_complete_csv[
            (group_1_fixes_complete_csv['Match'] == match) & 
            (group_1_fixes_complete_csv['Date'] == date) &
            (group_1_fixes_complete_csv['Nationality'] == nationality) &
            (group_1_fixes_complete_csv['Names_Found'].apply(lambda x: isinstance(x, list) and any(name in x for name in row['Names_Found'])))
        ]
    else:
        rows_matching_this_one = group_1_fixes_complete_csv[(group_1_fixes_complete_csv['Match'] == match) & 
                                        (group_1_fixes_complete_csv['Date'] == date) &
                                        (group_1_fixes_complete_csv['Nationality'] == nationality) &
                                        (group_1_fixes_complete_csv['Names_Found'] == row['Names_Found'])]
    num_identical_rows = len(rows_matching_this_one)
    num_names_this = len(names_found)

    if num_names_this != 1:
        if num_identical_rows == num_names_this:
            matching_indexes = list(rows_matching_this_one.index)
            num = 0
            number_of_changes += 1
            for matching_index in matching_indexes:
                print(f'index is {matching_index}, num is {num}')
                group_1_fixes_complete_csv.at[matching_index, 'Names_Found'] = names_found[num]
                print(f'added {names_found[num]} to row {matching_index}')
                num += 1

In [ ]:
for i in range(0, len(group_1_fixes_complete_csv)):
    if type(group_1_fixes_complete_csv.at[i, 'Names_Found']) != list:
        if group_1_fixes_complete_csv.at[i, 'Status'] == 'MULTIPLE':
            group_1_fixes_complete_csv.at[i, 'Status'] = 'Success'

for i in range(0, len(group_1_fixes_complete_csv)):
    if group_1_fixes_complete_csv.at[i, 'Status'] == 'Multi':
        if type(group_1_fixes_complete_csv.at[i, 'Names_Found']) == str:
            group_1_fixes_complete_csv.at[i, 'Status'] = 'Success'
        else:
            print(i)

In [ ]:
group_1_fixes_complete_csv.to_csv('Group1_fixes_complete_ii.csv')

## Checking rows w multiple names against the big dataset

In [ ]:
group_1_fixes_complete_csv_ii = load_csv_dataset('Group1_fixes_complete_ii.csv')
group_1_fixes_complete_csv_ii.at[5848, 'Names_Found'] = 'Ahmed Hamed Mahmoud'

In [ ]:
#loading datasets to look for stuff in 

#go through these datasets where the matching worked CORRECTLY & SEE IF THERES ONE OF THE NAMES IN THE MULTI SET.

#the big dataset to add stuff to 
combined_df_large = load_csv_dataset('Combined_DF_WORKING - ADD TO THIS.csv')
#terrells new df
terrell_new_df = load_csv_dataset('CSVs of edge cases - for T/Terrell Lookup Cases - new_df.csv')
#Cyrillic Found Guys
Cyrillic_Found_Guys = load_csv_dataset('Group 2 (for T) - Find their $ info/Cyrillic_Found_Guys.csv')
#lookup_required_matches
lookup_required_matches = load_csv_dataset('Group 2 (for T) - Find their $ info/lookup_required_matches.csv')
#Online lookup required - 1 name found
online_lookup_required_matches = load_csv_dataset('Group 2 (for T) - Find their $ info/Online lookup required - 1 name found.csv')


for i in range(0, len(lookup_required_matches)):
    if type(lookup_required_matches.at[i, 'Name Match']) == str:
        if (lookup_required_matches.at[i, 'Name Match'].startswith('[')):
            0==0
        else:
            #print(lookup_required_matches.at[i, 'Name Match'])
            lookup_required_matches.at[i, 'Name Match'] = f"[\'{lookup_required_matches.at[i, 'Name Match']}\']"

for i in range(0, len(lookup_required_matches)):
    if lookup_required_matches.at[i, "Name Match"] == '[\'Fabrice N\'Sakala\']':
        lookup_required_matches.at[i, "Name Match"] ='[\'Fabrice NSakala\']'
    if lookup_required_matches.at[i, "Name Match"] == '[\'Jordan N\'Kololo\']':
        lookup_required_matches.at[i, "Name Match"] ='[\'Jordan NKololo\']'

#check if there's an overlapping set of multiple dudes in other datasets as well 
    #Terrells new DF (already loaded)
    #big dataset (already loaded)
    #cyrillic found guys (already loaded)
    #online lookup - 1 name found (already loaded)
    #lookup required matches (already loaded)

    #other group
    #match date zero guys - Name(s) Found
match_date_zero_guys = load_csv_dataset('CSVs of edge cases - for F/"other" group/match_date_zero_guys.csv')
            #if the same list or if ur list is IN their list 
    #large dataset said 0
large_dataset_said_0 = load_csv_dataset('CSVs of edge cases - for F/"other" group/large_dataset_SAID_0.csv')
    #terrell's wrong DF 
terrell_wrong_df = load_csv_dataset('CSVs of edge cases - for F/"other" group/Terrell Lookup Cases - wrong_df.csv')


In [534]:
#csv files for third round of looking shit up
odd_name_0_cases = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/odd_name_found_0_cases.csv')
cyrillic_guys_0_found = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Cyrillic_Guys_Nobody_Found.csv')
online_lookup_guys_0_found = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Online lookup required - 0 names found.csv')


lookup_dudes = load_csv_dataset('CSVs of edge cases - for F/"other" group/lookup_required_dudes.csv')


In [546]:
instances = 0
still_working_counter = 0
success_counter = 0
second_part_success_counter = 0
single_mandem_counter = 0
for i in range(0, 5957):#0, 5957
    this_row_removals = 0
    case_found = False
    names_found = ''
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = group_1_fixes_complete_csv_ii.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']
    names_found_string = row['Names_Found']
    if '‘' in names_found_string:
        names_found_string = names_found_string.replace('‘', "\'")
        names_found_string = names_found_string.replace('’', "\'")

    original_jersey = row['ORIGINAL JERSEY']
    nationality = row['Nationality']
    if type(names_found_string) == str:
        if names_found_string.startswith('['):
            names_found = eval(names_found_string)
        else: 
            0==0
            #names_found = list(names_found_string)
    elif type(names_found_string) == list:
        names_found = names_found_string
    if len(names_found) >= 2: #1st ROUND CHECKING FOR SINGLES TO REMOVE
        #print(i, "Multiple names remaining", len(names_found), names_found, match, date, nationality)
        instances +=1

        #COMBINED DF LARGE
        combined_df_large_rows_matching = combined_df_large[(combined_df_large['Match'] == match) & 
                                                            (combined_df_large['Date'] == date) & 
                                                            ((combined_df_large['Name(s) Found'] == f"{names_found}") | (combined_df_large['ORIGINAL JERSEY'] == original_jersey) | ((combined_df_large['Name(s) Found'].apply(lambda x: isinstance(x, list) and any(name in x for name in row['Names_Found'])))))]
        if len(combined_df_large_rows_matching) >= 1:
            success_counter += 1
            print(i, 'Found', name_single, names_found)
            #remove the name you FOUND from names_found
            names_found.remove(name_single)
            this_row_removals += 1
            print(f"{i} - found using BIG dataset. names found is now {names_found}. removed {name_single}")
            #add the updated names_found to group_1_fixes_complete_csv_ii
            group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
            if len(names_found) == 1:
                #change the row in group_1_fixes_complete_csv_ii status to Success
                    group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
            #break this loop 
                #break
        for name_single in names_found:
            combined_df_large_rows_matching = combined_df_large[(combined_df_large['Match'] == match) & 
            (combined_df_large['Date'] == date) & 
            ((combined_df_large['Name(s) Found'] == name_single) | (combined_df_large['ORIGINAL JERSEY'] == original_jersey) | (combined_df_large['Name(s) Found'] == f"[{name_single}]"))]

            if len(combined_df_large_rows_matching) >= 1:
                success_counter += 1
                print(i, 'Found', name_single, names_found)
                #remove the name you FOUND from names_found
                names_found.remove(name_single)
                this_row_removals += 1
                print(f"{i} - found using BIG dataset. names found is now {names_found}. removed {name_single}")
                #add the updated names_found to group_1_fixes_complete_csv_ii
                group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
                if len(names_found) == 1:
                    #change the row in group_1_fixes_complete_csv_ii status to Success
                        group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
                #break this loop 
                    #break

        #NEW DF 
        new_df_rows_matching = terrell_new_df[(terrell_new_df['Match'] == match) & 
                                                            (terrell_new_df['Date'] == date) & 
                                                            ((terrell_new_df['Name(s) Found'] == f"{names_found}") | (terrell_new_df['ORIGINAL JERSEY'] == original_jersey) | ((terrell_new_df['Name(s) Found'].apply(lambda x: isinstance(x, list) and any(name in x for name in row['Names_Found'])))))]
        if len(new_df_rows_matching) >= 1:
            success_counter += 1
            print(i, 'Found', name_single, names_found)
            #remove the name you FOUND from names_found
            names_found.remove(name_single)
            this_row_removals += 1
            print(f"{i} - found using Terrell NEW DF. names found is now {names_found}. removed {name_single}")
            #add the updated names_found to group_1_fixes_complete_csv_ii
            group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
            if len(names_found) == 1:
                #change the row in group_1_fixes_complete_csv_ii status to Success
                    group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
            #break this loop 
                #break
        for name_single in names_found:
            new_df_rows_matching = terrell_new_df[(terrell_new_df['Match'] == match) & 
            (terrell_new_df['Date'] == date) & 
            ((terrell_new_df['Name(s) Found'] == name_single) | (terrell_new_df['ORIGINAL JERSEY'] == original_jersey) | (terrell_new_df['Name(s) Found'] == f"[{name_single}]"))]
            if len(new_df_rows_matching) >= 1:
                success_counter += 1
                print(i, 'Found', name_single, names_found)
                #remove the name you FOUND from names_found
                names_found.remove(name_single)
                this_row_removals += 1
                if name_single in names_found:
                    names_found.remove(name_single)
                print(f"{i} - found using Terrell NEW DF dataset. names found is now {names_found}. removed {name_single}")
                #add the updated names_found to group_1_fixes_complete_csv_ii
                group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
                if len(names_found) == 1:
                    #change the row in group_1_fixes_complete_csv_ii status to Success
                        group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
                #break this loop 
                    #break

        #Cyrillic Found guys
        cyrillic_Found_rows_matching = Cyrillic_Found_Guys[(Cyrillic_Found_Guys['Match'] == match) & 
                                                            (Cyrillic_Found_Guys['Date'] == date) & 
                                                            ((Cyrillic_Found_Guys['Match(es) Found'] == f"{names_found}") | (Cyrillic_Found_Guys['English Name'] == original_jersey) | ((Cyrillic_Found_Guys['Match(es) Found'].apply(lambda x: isinstance(x, list) and any(name in x for name in row['Names_Found'])))))]
        if len(cyrillic_Found_rows_matching) >= 1:
            success_counter += 1
            print(i, 'Found', name_single, names_found)
            #remove the name you FOUND from names_found
            names_found.remove(name_single)
            this_row_removals += 1
            print(f"{i} - found using CYRILLIC FOUND DF. names found is now {names_found}. removed {name_single}")
            #add the updated names_found to group_1_fixes_complete_csv_ii
            group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
            if len(names_found) == 1:
                #change the row in group_1_fixes_complete_csv_ii status to Success
                    group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
            #break this loop 
                #break
        for name_single in names_found:
            cyrillic_Found_rows_matching = Cyrillic_Found_Guys[(Cyrillic_Found_Guys['Match'] == match) & 
            (Cyrillic_Found_Guys['Date'] == date) & 
            (Cyrillic_Found_Guys.apply(lambda row: name_single in eval(row['Match(es) Found']), axis=1) | 
             (Cyrillic_Found_Guys['ORIGINAL JERSEY'] == original_jersey) | (Cyrillic_Found_Guys['Match(es) Found'] == f"[{name_single}]"))]
            #cyrillic_Found_rows_matching = Cyrillic_Found_Guys[(Cyrillic_Found_Guys['Match'] == match) & 
            #(Cyrillic_Found_Guys['Date'] == date) & 
            #((Cyrillic_Found_Guys['Name(s) Found'] == name_single) | (Cyrillic_Found_Guys['ORIGINAL JERSEY'] == original_jersey))]
            if len(cyrillic_Found_rows_matching) >= 1:
                #print(f'name is {name_single}. len is {len(cyrillic_Found_rows_matching)}. index is {cyrillic_Found_rows_matching.index}')
                success_counter += 1
                print(i, 'Found', name_single, names_found)
                #remove the name you FOUND from names_found
                names_found.remove(name_single)
                this_row_removals += 1
                print(f"{i} - found using CYRILLIC FOUND DF dataset. names found is now {names_found}. removed {name_single}")
                #add the updated names_found to group_1_fixes_complete_csv_ii
                group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
                if len(names_found) == 1:
                    #change the row in group_1_fixes_complete_csv_ii status to Success
                       group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
                #break this loop 
                    #break
        #lookup req'd matches 
        lookup_reqd_rows_matching = lookup_required_matches[(lookup_required_matches['Match'] == match) & 
                                                            (lookup_required_matches['Date'] == date) & 
                                                            ((lookup_required_matches['Name Match'] == f"{names_found}") | (lookup_required_matches['Jersey Name'] == original_jersey) | ((lookup_required_matches['Name Match'].apply(lambda x: isinstance(x, list) and any(name in x for name in row['Names_Found'])))))]
        if len(lookup_reqd_rows_matching) >= 1:
            success_counter += 1
            print(i, 'Found', name_single, names_found)
            #remove the name you FOUND from names_found
            names_found.remove(name_single)
            this_row_removals += 1
            print(f"{i} - found using LOOKUP REQD DF. names found is now {names_found}. removed {name_single}")
            #add the updated names_found to group_1_fixes_complete_csv_ii
            group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
            if len(names_found) == 1:
                #change the row in group_1_fixes_complete_csv_ii status to Success
                    group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
            #break this loop 
                #break
        for name_single in names_found:
            lookup_reqd_rows_matching = lookup_required_matches[(lookup_required_matches['Match'] == match) & 
            (lookup_required_matches['Date'] == date) & 
            (lookup_required_matches.apply(lambda row_1: name_single in eval(row_1['Name Match']), axis=1) | 
             (lookup_required_matches['Jersey Name'] == original_jersey))]
            #cyrillic_Found_rows_matching = Cyrillic_Found_Guys[(Cyrillic_Found_Guys['Match'] == match) & 
            #(Cyrillic_Found_Guys['Date'] == date) & 
            #((Cyrillic_Found_Guys['Name(s) Found'] == name_single) | (Cyrillic_Found_Guys['ORIGINAL JERSEY'] == original_jersey))]
            if len(lookup_reqd_rows_matching) >= 1:
                #print(f'name is {name_single}. len is {len(cyrillic_Found_rows_matching)}. index is {cyrillic_Found_rows_matching.index}')
                success_counter += 1
                print(i, 'Found', name_single, names_found)
                #remove the name you FOUND from names_found
                names_found.remove(name_single)
                this_row_removals += 1
                print(f"{i} - found using LOOKUP REQD dataset. names found is now {names_found}. removed {name_single}")
                #add the updated names_found to group_1_fixes_complete_csv_ii
                group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
                if len(names_found) == 1:
                    #change the row in group_1_fixes_complete_csv_ii status to Success
                        group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
                #break this loop 
                    #break

        
        #online lookup req'd 1 name found
        online_lookup_required_df_large_rows_matching = online_lookup_required_matches[(online_lookup_required_matches['Match'] == match) & 
                                                            (online_lookup_required_matches['Date'] == date) & 
                                                            ((online_lookup_required_matches['Name(s) Found'] == f"{names_found}") | (online_lookup_required_matches['ORIGINAL JERSEY'] == original_jersey) | ((online_lookup_required_matches['Name(s) Found'].apply(lambda x: isinstance(x, list) and any(name in x for name in row['Names_Found'])))))]
        if len(online_lookup_required_df_large_rows_matching) >= 1:
            success_counter += 1
            print(i, 'Found', name_single, names_found)
            #remove the name you FOUND from names_found
            names_found.remove(name_single)
            this_row_removals += 1
            print(f"{i} - found using ONLINE LOOKUP REQD DF. names found is now {names_found}. removed {name_single}")
            #add the updated names_found to group_1_fixes_complete_csv_ii
            group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
            if len(names_found) == 1:
                #change the row in group_1_fixes_complete_csv_ii status to Success
                    group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
            #break this loop 
                #break
        for name_single in names_found:
            online_lookup_required_df_large_rows_matching = online_lookup_required_matches[(online_lookup_required_matches['Match'] == match) & 
            (online_lookup_required_matches['Date'] == date) & 
            ((online_lookup_required_matches['Name(s) Found'] == name_single) | (online_lookup_required_matches['ORIGINAL JERSEY'] == original_jersey) | (online_lookup_required_matches['Name(s) Found'] == f"[{name_single}]"))]

            if len(online_lookup_required_df_large_rows_matching) >= 1:
                success_counter += 1
                print(i, 'Found', name_single, names_found)
                #remove the name you FOUND from names_found
                names_found.remove(name_single)
                this_row_removals += 1
                print(f"{i} - found using ONLINE LOOKUP REQUIRED dataset. names found is now {names_found}. removed {name_single}")
                #add the updated names_found to group_1_fixes_complete_csv_ii
                group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found
                if len(names_found) == 1:
                    #change the row in group_1_fixes_complete_csv_ii status to Success
                        group_1_fixes_complete_csv_ii.at[i, 'Status'] = 'Success'
                #break this loop 
                    #break
    if len(names_found) >= 2: #SECOND ROUND checking for similar arrays
        if this_row_removals != 0:
            0==0
        else:
            still_working_counter += 1
        #if still multiple:
        #check if there's an overlapping set of multiple dudes in other datasets as well 

            #if theres a matching multi set, assign one name to each row.

        #combined DF large
        name_conditions = [combined_df_large['Name(s) Found'].str.contains(name) for name in names_found]
        combined_condition = reduce(lambda x, y: x & y, name_conditions)
        final_condition = (
            (combined_df_large['Date'] == date) & 
            (combined_df_large['Match'] == match) & 
            (
                (combined_df_large['Name(s) Found'] == f"{names_found}") | 
                combined_condition
            )
        )
        combined_df_large_match = combined_df_large[final_condition]
        if len(combined_df_large_match) >= 1:
            print(i, 'xxxxx - found a matching set of names in combined DF', names_found, combined_df_large_match['Name(s) Found'].unique())
            second_part_success_counter += 1
            still_working_counter -= 1

        #terrell new DF 
        name_conditions = [terrell_new_df['Name(s) Found'].str.contains(name) for name in names_found]
        combined_condition = reduce(lambda x, y: x & y, name_conditions)
        final_condition = (
            (terrell_new_df['Date'] == date) & 
            (terrell_new_df['Match'] == match) & 
            (
                (terrell_new_df['Name(s) Found'] == f"{names_found}") | 
                combined_condition
            )
        )
        t_new_df_match = terrell_new_df[final_condition]
        if len(t_new_df_match) >= 1:
            print(i, 'xxxxx - found a matching set of names in terrells new DF', names_found, t_new_df_match['Name(s) Found'].unique())
            second_part_success_counter += 1
            still_working_counter -= 1

        #large Dataset said 0 - Name(s) Found column
        name_conditions = [large_dataset_said_0['Name(s) Found'].str.contains(name) for name in names_found]
        combined_condition = reduce(lambda x, y: x & y, name_conditions)
        final_condition = (
            (large_dataset_said_0['Date'] == date) & 
            (large_dataset_said_0['Match'] == match) & 
            (
                (large_dataset_said_0['Name(s) Found'] == f"{names_found}") | 
                combined_condition
            )
        )
        large_dataset_0_match = large_dataset_said_0[final_condition]
        if len(large_dataset_0_match) >= 1:
            print(i, 'xxxxx - found a matching set of names in the large dataset which said that 0', names_found, large_dataset_0_match['Name(s) Found'].unique())
            second_part_success_counter += 1
            still_working_counter -= 1
            if len(names_found) == len(eval(large_dataset_0_match['Name(s) Found'].unique()[0])):
                if len(names_found) == 2:
                    large_dataset_said_0.at[large_dataset_0_match.index[0], 'Name(s) Found'] = names_found[0]
                    group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found[1]
                else:
                    print(len(names_found), 'oddity')
        
        #match date 0 guys - Name column 
        name_conditions = [match_date_zero_guys['Name'].str.contains(name) for name in names_found]
        combined_condition = reduce(lambda x, y: x & y, name_conditions)
        final_condition = (
            (match_date_zero_guys['Date'] == date) & 
            (match_date_zero_guys['Match'] == match) & 
            (
                (match_date_zero_guys['Name'] == f"{names_found}") | 
                combined_condition
            )
        )
        match_date_0_match = match_date_zero_guys[final_condition]
        if len(match_date_0_match) >= 1:
            print(i, 'xxxxx - found a matching set of names in the match date filter left 0 dataset', names_found, match_date_0_match['Name'].unique())
            second_part_success_counter += 1
            still_working_counter -= 1
            if len(names_found) == len(eval(match_date_0_match['Name'].unique()[0])):
                if len(names_found) == 2:
                    match_date_zero_guys.at[match_date_0_match.index[0], 'Name'] = names_found[0]
                    group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found[1]
                else:
                    print(len(names_found), 'oddity')

        #wrong DF - Found Name column
        name_conditions = [terrell_wrong_df['Found Name'].str.contains(name) for name in names_found]
        combined_condition = reduce(lambda x, y: x & y, name_conditions)
        final_condition = (
            (terrell_wrong_df['Date'] == date) & 
            (terrell_wrong_df['Match'] == match) & 
            (
                (terrell_wrong_df['Found Name'] == f"{names_found}") | 
                combined_condition
            )
        )
        wrong_df_match = terrell_wrong_df[final_condition]
        if len(wrong_df_match) >= 1:
            print(i, 'xxxxx - found a matching set of names in Terrell\'s wrong DF', names_found, wrong_df_match['Found Name'].unique())
            second_part_success_counter += 1
            still_working_counter -= 1
            if len(names_found) == len(eval(wrong_df_match['Found Name'].unique()[0])):
                if len(names_found) == 2:
                    terrell_wrong_df.at[wrong_df_match.index[0], 'Found Name'] = names_found[0]
                    group_1_fixes_complete_csv_ii.at[i, 'Names_Found'] = names_found[1]
                else:
                    print(len(names_found), 'oddity')

    if len(names_found) >= 2: #3 check for empty rows w the same jersey
        print(f"***{i}***")
        odd_name_candidates = odd_name_0_cases[(odd_name_0_cases['Match'] == match) &
                 (odd_name_0_cases['Date'] == date) & 
                 (odd_name_0_cases['Nationality'] == nationality)]
        
        if len(odd_name_candidates) >= 1:
             print(i, 'found a row of this in odd candidates')

        cyrillic_row_candidates = cyrillic_guys_0_found[(cyrillic_guys_0_found['Match'] == match) &
                        (cyrillic_guys_0_found['Date'] == date) & 
                 (cyrillic_guys_0_found['Nationality'] == nationality)]
        
        if len(cyrillic_row_candidates) >= 1:
             print(i, 'found a row of this in cyrillic candidates')

        online_lookup_row_candidates = online_lookup_guys_0_found[(online_lookup_guys_0_found['Match'] == match) &
                        (online_lookup_guys_0_found['Date'] == date) & 
                 (online_lookup_guys_0_found['Nationality'] == nationality)]
        
        if len(online_lookup_row_candidates) >= 1:
             print(i, 'Found a row of this in online lookup guys')

        lookup_dudes_candidates = lookup_dudes[(lookup_dudes['Match'] == match) & 
                                               (lookup_dudes['Date'] == date) & 
                                               (lookup_dudes['Nationality'] == nationality)]
        
        if len(lookup_dudes_candidates) >= 1:
             print(i, 'found a candidate rrow in weird lookup dudes')


        
print(instances, success_counter, still_working_counter, second_part_success_counter, single_mandem_counter)


#STILL MULTIPLE

#check across the weird datasets to find this jersey, match and date but a FAIL

    #if theres 1 row with the right looking jersey and 0 for a name
        #and you have 2 names in the set

    #same if you find 2 rows with a 0 and there are 3 names in the set 

    #group 3
        #odd_name_found_0_cases
        #Cyrillic Guys Nobody Found
        #Online lookup required - 0 names found

    #other group
        #lookup_required_dudes



#prev run - 271 instances, 64 success. 191 still working. 16 success in part 2
#2nd - 212 instances, 0 success. 196 still working. 16 success in part 2
#3rd - 204 instances, 0 success. 196 still working. 8 success in part 2


#figure out if there are 2 names in names found
#and 2 names in the array it found.

#if so, assign 1 name to each dataset. 

***81***
***99***
99 found a candidate rrow in weird lookup dudes
***395***
395 found a row of this in odd candidates
***526***
***530***
***535***
***540***
***586***
586 found a candidate rrow in weird lookup dudes
***625***
***769***
***804***
986 xxxxx - found a matching set of names in the match date filter left 0 dataset ['Alexander López', 'Luis López'] ["['Luis López', 'Alexander López', 'Iván López', 'Kevin López', 'Roberto López']"]
***986***
***1022***
***1110***
***1150***
***1181***
***1255***
***1320***
***1573***
1605 xxxxx - found a matching set of names in the match date filter left 0 dataset ['Luis López', 'Alexander López'] ["['Luis López', 'Alexander López', 'Iván López', 'Kevin López', 'Roberto López']"]
***1605***
1620 xxxxx - found a matching set of names in the match date filter left 0 dataset ['Luis López', 'Alexander López'] ["['Luis López', 'Alexander López', 'Iván López', 'Kevin López', 'Roberto López']"]
***1620***
***1713***
***1765***
***1771***
***1956**

In [553]:
#group_1_fixes_complete_csv_ii.to_csv('updated_Group1_fixes_complete_ii.csv')
#large_dataset_said_0.to_csv('updated_large_dataset_said_0.csv')
#combined_df_large.to_csv('updated_large_combined_DF.csv')
#terrell_wrong_df.to_csv('updated Terrell lookup cases - wrong DF.csv')
#match_date_zero_guys.to_csv('updated match date said zero guys.csv')
#terrell_new_df.to_csv('updated Terrell new DF.csv')